In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
import io
import datetime
import matplotlib.dates as mdates

In [2]:
from sys import platform
if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/green/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/green/"

In [19]:
year = 2017
csv_name = data_string+str(year)+"_green_all_R_new.csv"
df = pd.read_csv(csv_name, header=0)
df['midpoint'] = pd.to_datetime(df['midpoint'])

df['Date'] = pd.to_datetime(df['start_datetime']).dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['doy'] = df['Date'].dt.dayofyear



In [20]:
#checking whether the midpoint of the time when trap was in during day or night
def categorize_daytime(dt):
    if dt.hour >= 6 and dt.hour < 18:
        return 'Day'
    else:
        return 'Night'
df['daytime_category'] = df['midpoint'].apply(categorize_daytime)

In [21]:
df['chinook0_wild_perhour'] = df['chinook0_wild_num']/df['In']
df['chinook0_hatchery_perhour'] = df['chinook0_hatchery_num']/df['In']

df['coho0_wild_perhour'] = df['coho0_wild_num']/df['In']
df['chinook1_hatchery_perhour'] = df['chinook1_hatchery_num']/df['In']

df['coho1_wild_perhour'] = df['coho1_wild_num']/df['In']
df['coho1_hatchery_perhour'] = df['coho1_hatchery_num']/df['In']

df['steelheadsmolt_wild_perhour'] = df['steelheadsmolt_wild_num']/df['In']
df['steelheadsmolt_hatchery_perhour'] = df['steelheadsmolt_hatchery_num']/df['In']

df['chum0_mixed_perhour'] = df['chum0_mixed_num']/df['In']

In [22]:
df_day = df[df['daytime_category']=="Day"]
df_night = df[df['daytime_category']=="Night"]

In [25]:
#need to have one row for every date
df_dates_day = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2017-01-17 12:00:00', 
                                       end='2017-06-29 12:00:00')})
df_dates_night = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2017-01-17 00:00:00', 
                                       end='2017-06-29 00:00:00')})

df_merged_day = pd.merge_asof(df_dates_day, 
              df_day, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

df_merged_night = pd.merge_asof(df_dates_night, 
              df_night, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

In [29]:
#doy columns cannot be nan, so making those columns again
#I will keep the current doy in midpoint doy 
#daytime_category columns cannot be nan
df_merged_day['midpoint_doy'] = df_merged_day['doy']
df_merged_day['doy'] = df_merged_day['date_range'].dt.dayofyear
df_merged_day['year'] = df_merged_day['date_range'].dt.year
df_merged_day['daytime_category'] = 'day'

df_merged_night['midpoint_doy'] = df_merged_night['doy']
df_merged_night['doy'] = df_merged_night['date_range'].dt.dayofyear
df_merged_night['year'] = df_merged_night['date_range'].dt.year
df_merged_night['daytime_category'] = 'night'

In [30]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_merged_night)

,date_range,Unnamed: 0,start_datetime,end_datetime,In,midpoint,StartDate,EndDate,chum0_mixed_num,steelheadsmolt_wild_num,steelheadsmolt_hatchery_num,coho1_wild_num,coho1_hatchery_num,coho0_wild_num,chinook1_hatchery_num,chinook0_wild_num,chinook0_hatchery_num,Date,doy,daytime_category,chinook0_wild_perhour,chinook0_hatchery_perhour,coho0_wild_perhour,chinook1_hatchery_perhour,coho1_wild_perhour,coho1_hatchery_perhour,steelheadsmolt_wild_perhour,steelheadsmolt_hatchery_perhour,chum0_mixed_perhour,midpoint_doy,year
0,2017-01-17,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,17,night,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,2017
1,2017-01-18,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,18,night,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,2017
2,2017-01-19,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,19,night,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,2017
3,2017-01-20,3.0,2017-01-19 15:00:00,2017-01-20 08:30:00,17.500000,2017-01-19 23:45:00,2017-01-19,2017-01-20,0.0,0.0,0.0,13.0,0.0,0.0,0.0,35.0,0.0,2017-01-19,20,night,2.000000,0.000000,0.000000,0.000000,0.742857,0.000000,0.000000,0.000000,0.000000,20,2017
4,2017-01-21,5.0,2017-01-20 15:00:00,2017-01-21 08:30:00,17.500000,2017-01-20 23:45:00,2017-01-20,2017-01-21,0.0,5.0,0.0,13.0,0.0,0.0,0.0,43.0,0.0,2017-01-20,21,night,2.457143,0.000000,0.000000,0.000000,0.742857,0.000000,0.285714,0.000000,0.000000,21,2017
5,2017-01-22,7.0,2017-01-21 15:00:00,2017-01-22 08:30:00,17.500000,2017-01-21 23:45:00,2017-01-21,2017-01-22,0.0,1.0,0.0,8.0,0.0,0.0,0.0,31.0,0.0,2017-01-21,22,night,1.771429,0.000000,0.000000,0.000000,0.457143,0.000000,0.057143,0.000000,0.000000,22,2017
6,2017-01-23,9.0,2017-01-22 16:00:00,2017-01-23 08:00:00,16.000000,2017-01-23 00:00:00,2017-01-22,2017-01-23,0.0,2.0,0.0,1.0,0.0,0.0,0.0,41.0,0.0,2017-01-22,23,night,2.562500,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,23,2017
7,2017-01-24,11.0,2017-01-23 16:00:00,2017-01-24 08:00:00,16.000000,2017-01-24 00:00:00,2017-01-23,2017-01-24,0.0,0.0,0.0,3.0,0.0,0.0,0.0,50.0,0.0,2017-01-23,24,night,3.125000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,24,2017
8,2017-01-25,13.0,2017-01-24 16:00:00,2017-01-25 08:00:00,16.000000,2017-01-25 00:00:00,2017-01-24,2017-01-25,0.0,0.0,0.0,4.0,0.0,0.0,0.0,23.0,0.0,2017-01-24,25,night,1.437500,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,25,2017
9,2017-01-26,15.0,2017-01-25 16:00:00,2017-01-26 08:00:00,16.000000,2017-01-26 00:00:00,2017-01-25,2017-01-26,0.0,0.0,0.0,5.0,0.0,1.0,0.0,44.0,0.0,2017-01-25,26,night,2.750000,0.000000,0.062500,0.000000,0.312500,0.000000,0.000000,0.000000,0.000000,26,2017


In [31]:
#save csv

df_merged_day.to_csv(data_string + "green_2017_day.csv")
df_merged_night.to_csv(data_string + "green_2017_night.csv")